<a href="https://colab.research.google.com/github/krimits/trifon/blob/main/Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Βήμα 1: Εγκατάσταση Απαραίτητων Βιβλιοθηκών
# Αυτή η εντολή λειτουργεί άψογα στο Google Colab.
!pip install transformers datasets torch pandas scikit-learn

import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

# Βήμα 2: Φόρτωση και Προετοιμασία του Dataset
print("\n--- Βήμα 2: Φόρτωση και Προετοιμασία του Dataset ---")
# Φορτώνουμε ένα δείγμα δεδομένων από το Hugging Face Hub με κριτικές προϊόντων.
# Αντικαθιστούμε το προηγούμενο dataset με το 'dair-ai/emotion' που είναι διαθέσιμο.
try:
    dataset = load_dataset("dair-ai/emotion")
    print("Το dataset φορτώθηκε με επιτυχία.")
except Exception as e:
    print(f"Σφάλμα κατά τη φόρτωση του dataset: {e}")
    # Αν αποτύχει η φόρτωση, μπορείτε να προσθέσετε κώδικα για να φορτώσετε ένα τοπικό αρχείο ή να σταματήσετε την εκτέλεση.
    # Για τώρα, θα σταματήσουμε την εκτέλεση αν το dataset δεν φορτωθεί.
    raise

# Το dataset 'dair-ai/emotion' έχει ήδη split σε train/validation/test.
# Θα χρησιμοποιήσουμε train και test splits.
dataset_dict = DatasetDict({
    'train': dataset['train'],
    'test': dataset['test']
})


print("\nΕπεξεργασμένο dataset για ταξινόμηση συναισθήματος:")
print(dataset_dict)
print("\nΠαράδειγμα από το training set:")
print(dataset_dict['train'][0])

# Βήμα 3: Προ-επεξεργασία (Tokenization)
print("\n--- Βήμα 3: Προ-επεξεργασία (Tokenization) ---")
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(f"Tokenizer για το μοντέλο '{model_name}' φορτώθηκε.")

def tokenize_function(examples):
    # Ο tokenizer μετατρέπει το κείμενο σε αριθμούς (input IDs, attention mask).
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# Εφαρμόζουμε τη συνάρτηση tokenization σε ολόκληρο το dataset.
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)
print("Το dataset έχει μετατραπεί σε tokenized μορφή.")

# Βήμα 4: Φόρτωση Προ-εκπαιδευμένου Μοντέλου & Fine-Tuning
print("\n--- Βήμα 4: Φόρτωση Προ-εκπαιδευμένου Μοντέλου & Fine-Tuning ---")
# Φορτώνουμε το μοντέλο DistilBERT, έτοιμο για ταξινόμηση κειμένου.
# Το dataset 'dair-ai/emotion' έχει 6 ετικέτες, οπότε num_labels=6.
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)
print(f"Μοντέλο '{model_name}' φορτώθηκε.")

# Ορίζουμε τις παραμέτρους για την εκπαίδευση (π.χ. epochs, batch size).
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1, # Μείωση των epochs για ταχύτερη εκτέλεση στο παράδειγμα
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    eval_strategy="epoch", # Changed from evaluation_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    # Ενεργοποίηση GPU αν είναι διαθέσιμο
    no_cuda=not torch.cuda.is_available()
)

# Ορίζουμε μια συνάρτηση που θα υπολογίζει τις μετρικές απόδοσης (accuracy, F1-score, κ.λπ.).
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Χρησιμοποιούμε average='weighted' ή None για multi-class classification
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Δημιουργούμε το αντικείμενο Trainer που θα διαχειριστεί την εκπαίδευση.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

print("\nΞεκινάει η διαδικασία του Fine-Tuning...")
trainer.train()
print("Το Fine-Tuning ολοκληρώθηκε.")

# Βήμα 5: Αξιολόγηση του Μοντέλου
print("\n--- Βήμα 5: Αξιολόγηση του Μοντέλου ---")
evaluation_results = trainer.evaluate()
print("Αποτελέσματα Αξιολόγησης στο test set:")
for key, value in evaluation_results.items():
    print(f"{key.replace('eval_', '').capitalize():<12}: {value:.4f}")

model_save_path = "./fine_tuned_emotion_model"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"\nΤο fine-tuned μοντέλο και ο tokenizer αποθηκεύτηκαν στο φάκελο '{model_save_path}'.")

# Βήμα 6: Πρόβλεψη σε Νέα Δεδομένα (Inference)
print("\n--- Βήμα 6: Πρόβλεψη σε Νέα Δεδομένα (Inference) ---")
new_reviews = [
    "Αισθάνομαι πολύ χαρούμενος σήμερα!",
    "Αυτό το προϊόν είναι απαίσιο.",
    "Είμαι τόσο λυπημένος και απογοητευμένος.",
    "Ήταν μια ευχάριστη έκπληξη!",
    "Το προιόν είναι καταπληκτικό"


]

# Ο χάρτης ετικετών για το dataset 'dair-ai/emotion'
labels_map = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}


for review in new_reviews:
    inputs = tokenizer(review, return_tensors="pt", padding=True, truncation=True, max_length=128)
    # Μεταφορά inputs στη συσκευή του μοντέλου (CPU ή GPU)
    if torch.cuda.is_available():
        inputs = {key: val.to('cuda') for key, val in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    print(f"\nΚριτική: '{review}'")
    print(f"Πρόβλεψη: {labels_map[predicted_class_id]}")


--- Βήμα 2: Φόρτωση και Προετοιμασία του Dataset ---
Το dataset φορτώθηκε με επιτυχία.

Επεξεργασμένο dataset για ταξινόμηση συναισθήματος:
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

Παράδειγμα από το training set:
{'text': 'i didnt feel humiliated', 'label': 0}

--- Βήμα 3: Προ-επεξεργασία (Tokenization) ---
Tokenizer για το μοντέλο 'distilbert-base-uncased' φορτώθηκε.


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Το dataset έχει μετατραπεί σε tokenized μορφή.

--- Βήμα 4: Φόρτωση Προ-εκπαιδευμένου Μοντέλου & Fine-Tuning ---


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Μοντέλο 'distilbert-base-uncased' φορτώθηκε.

Ξεκινάει η διαδικασία του Fine-Tuning...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.211100,0.170369,0.926000,0.925709,0.926788,0.926000


Το Fine-Tuning ολοκληρώθηκε.

--- Βήμα 5: Αξιολόγηση του Μοντέλου ---


Αποτελέσματα Αξιολόγησης στο test set:
Loss        : 0.1704
Accuracy    : 0.9260
F1          : 0.9257
Precision   : 0.9268
Recall      : 0.9260
Runtime     : 6.9530
Samples_per_second: 287.6450
Steps_per_second: 17.9780
Epoch       : 1.0000

Το fine-tuned μοντέλο και ο tokenizer αποθηκεύτηκαν στο φάκελο './fine_tuned_emotion_model'.

--- Βήμα 6: Πρόβλεψη σε Νέα Δεδομένα (Inference) ---

Κριτική: 'Αισθάνομαι πολύ χαρούμενος σήμερα!'
Πρόβλεψη: anger

Κριτική: 'Αυτό το προϊόν είναι απαίσιο.'
Πρόβλεψη: joy

Κριτική: 'Είμαι τόσο λυπημένος και απογοητευμένος.'
Πρόβλεψη: joy

Κριτική: 'Ήταν μια ευχάριστη έκπληξη!'
Πρόβλεψη: anger

Κριτική: 'Το προιόν είναι καταπληκτικό'
Πρόβλεψη: joy


In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Chahnwoo/distilbert_dair-ai_emotion_20240730_e200_cos")
model = AutoModelForSequenceClassification.from_pretrained("Chahnwoo/distilbert_dair-ai_emotion_20240730_e200_cos")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/872 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

### Χρήση ενός Προ-εκπαιδευμένου Μοντέλου για Πρόβλεψη

Μπορείτε να χρησιμοποιήσετε ένα προ-εκπαιδευμένο μοντέλο απευθείας από το Hugging Face Hub για να κάνετε προβλέψεις, χωρίς να χρειάζεται να κάνετε fine-tuning. Το παρακάτω κελί φορτώνει το μοντέλο `Chahnwoo/distilbert_dair-ai_emotion_20240730_e200_cos` και το χρησιμοποιεί για να προβλέψει τα συναισθήματα για τις προτάσεις στη λίστα `new_reviews`.

In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Φόρτωση του συγκεκριμένου προ-εκπαιδευμένου μοντέλου και tokenizer
model_name_pretrained = "Chahnwoo/distilbert_dair-ai_emotion_20240730_e200_cos"
tokenizer_pretrained = AutoTokenizer.from_pretrained(model_name_pretrained)
model_pretrained = AutoModelForSequenceClassification.from_pretrained(model_name_pretrained)

print(f"Προ-εκπαιδευμένο μοντέλο '{model_name_pretrained}' φορτώθηκε.")

# Ο χάρτης ετικετών για το dataset 'dair-ai/emotion'
labels_map = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

# Λίστα με νέα παραδείγματα για πρόβλεψη (χρησιμοποιούμε την ίδια λίστα new_reviews)
# new_reviews list is already defined in the previous cell

print("\n--- Πρόβλεψη με το Προ-εκπαιδευμένο Μοντέλο ---")
for review in new_reviews:
    inputs = tokenizer_pretrained(review, return_tensors="pt", padding=True, truncation=True, max_length=128)
    # Μεταφορά inputs στη συσκευή του μοντέλου (CPU ή GPU)
    if torch.cuda.is_available():
        inputs = {key: val.to('cuda') for key, val in inputs.items()}
        model_pretrained.to('cuda') # Μεταφορά του μοντέλου στην GPU

    with torch.no_grad():
        logits = model_pretrained(**inputs).logits
    predicted_class_id = logits.argmax().item()
    print(f"\nΚριτική: '{review}'")
    print(f"Πρόβλεψη: {labels_map[predicted_class_id]}")

Προ-εκπαιδευμένο μοντέλο 'Chahnwoo/distilbert_dair-ai_emotion_20240730_e200_cos' φορτώθηκε.

--- Πρόβλεψη με το Προ-εκπαιδευμένο Μοντέλο ---

Κριτική: 'Αισθάνομαι πολύ χαρούμενος σήμερα!'
Πρόβλεψη: anger

Κριτική: 'Αυτό το προϊόν είναι απαίσιο.'
Πρόβλεψη: joy

Κριτική: 'Είμαι τόσο λυπημένος και απογοητευμένος.'
Πρόβλεψη: joy

Κριτική: 'Ήταν μια ευχάριστη έκπληξη!'
Πρόβλεψη: joy
